I received some excel files from a retired geochemist containing information about electricity consumption in Ontario. He looked up the data [on the web](http://www.webroots.ca/static/ontarioelectricity/ontarioelectricity.html) three times daily and manually entered it into a spreadsheet.  There are three files in xlsx format. File 1 is observations from 2018, file 2 is observations from 2019, and file 3 is 1000 observations starting from August 2018 (approximately 13 months).

Loaded files in LibreOffice Calc to preview data. 

Most date entries are in dd-mmm format but a few are in dd-mmm-yy.

Column headers span two rows: row 1 is name and row 2 is units. Header in file 2 is an image instead of text.

File 3 contains two sheets with data. Looks like 1st sheet is a copy of the 2018 data and 2nd sheet is combined data from 2018-08-25 through 2019-09-30.

I see at least one data entry error (cell F12 in file 1).

Files 1 and 3 contain summary statisitics:
Three rows of stats at the bottom of the sheet: min, max, and avg of each column.
The two rightmost columns show average daily totals per month, and name of the month.

![](images/spreadsheet.png)

Fixed headers to contain both name and units in one row.

Set the format of all the date columns to yyyy-mm-dd.

Deleted summary statistics.

Exported all four sheets as CSV.

Diffed the exported csv files. The extra sheet in file 3 is an unchanged copy of the data from file 1. Generated checksums to verify.

In [1]:
!diff data/electricity1.csv data/electricity3a.csv
!cksum data/*.csv

2600525449 28136 data/electricity1.csv
465144111 96733 data/electricity2.csv
2815554452 102371 data/electricity3.csv
2600525449 28136 data/electricity3a.csv
976740961 131569 data/electricity4.csv


In [2]:
import pandas as pd
pd.options.display.max_columns = 40

In [3]:
df1 = pd.read_csv('data/electricity1.csv')
df2 = pd.read_csv('data/electricity2.csv')
df3 = pd.read_csv('data/electricity3.csv')

In [4]:
df4 = df1.append(df2)
df4.to_csv('data/electricity4.csv', index=False)